In [1]:
#import modul
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API 
from tweepy import Cursor
from tweepy import Stream
from textblob import TextBlob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [2]:
# Akses key token dan consumer key token untuk menghubungkan twitter dengan jupyter lab
ACCESS_TOKEN = "972856602377076736-9n9KJdQlm6RPlD8tLrTBOnT2wZRSdZU"
ACCESS_TOKEN_SECRET = "5mxCT0O3efmmHtmABpJH30BbzjhE1tZSUtvljUjuaCknG"
CONSUMER_KEY = "WdEkXvesactnxwzuTJlDRCQ2X"
CONSUMER_SECRET = "oK9bJkG7gF7PNKpM8t8mVAtYOl3YwNhfy5mkIB1PbOYnTdEyPZ"

In [3]:
# Client Twitter
class TwitterClient():
    def __init__(self, user_twitter=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.user_twitter = user_twitter

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.user_twitter).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.user_twitter).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.user_twitter).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

In [4]:
#Autentifikasi twitter
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

In [4]:
#Stream Twitter
class TwitterStreamer():
    """
    Class untuk streaming dan pemrosesan tweet langsung.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        #Autentifikasi twitter dan koneksi ke twitter streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        stream = Stream(auth, listener)

        # Untuk memfilter streaming twitter dengan hastag atau kata kunci
        stream.filter(track=hash_tag_list)

In [6]:
# Twitter Stream Listener
import tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
class TweetAnalyzer():
    """
    Fungsi untuk menganalisis tweet dan mengkategorikan konten-konten dari twitter.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

     #Menampilkan hasil kedalam sebuah tabel
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=["tweets"])
        df["No"] = np.arange(start = 1, stop = len(df)+1)
        df["Id"] = np.array([tweet.id for tweet in tweets])
        df["Date"] = np.array([tweet.created_at for tweet in tweets])
        df["Likes"] = np.array([tweet.favorite_count for tweet in tweets])
        df["Retweets"] = np.array([tweet.retweet_count for tweet in tweets])
        df = df[["No", "tweets", "Id", "Date", "Likes", "Retweets"]]
        return df

 
if __name__ == '__main__':

    tweet_analyzer = TweetAnalyzer()

    api = tweepy.API(auth)
    
    #streaming data dengan menggunakan hastag love dengan data yg di stream sebanyak 100 
    tweets = api.search(q=['#love'], count=100)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df["sentiment"] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df["tweets"]])

In [8]:
# meanmpilkan dataframe hasil streaming dengan hastag love
df.head()

,No,tweets,Id,Date,Likes,Retweets,sentiment
0,1,RT @steph_readings: So in love with my new dec...,1173075935756767232,2019-09-15 03:27:52,0,5,1
1,2,RT @svufanblog4015: The pure love and happines...,1173075923454889985,2019-09-15 03:27:49,0,4,1
2,3,#aspirations #goals #chemistry #motivation #dr...,1173075912981696513,2019-09-15 03:27:47,0,0,1
3,4,They’re ▪️🅱️🅰️©️K\n➖➖➖➖➖➖➖➖➖➖➖\n#koderedd #stl...,1173075843712552961,2019-09-15 03:27:30,0,0,0
4,5,RT @AnneScottlin: ❦Contrary to our frequent at...,1173075812989448197,2019-09-15 03:27:23,0,36,1
